In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score


In [13]:
from sklearn import preprocessing

In [14]:
all_data = df = pd.read_csv("all_data.csv")

In [15]:
all_data.head(90)

,time (UTC),battery (V),dewPoint (°C),humidity (%),irradiation (W/m2),irradiation_max (W/m2),pressure_raw (hPa),rain (mm),temperature (°C),temperature_max (°C),temperature_min (°C),rainfall_rate_max (mm/h),pressure (hPa),temperature_wetbulb_stull2011_C (°C)
0,2021-06-24 09:20:00,4.15,20.81,59.6,1438,1438,978.80,NaN,29.5,29.5,29.5,NaN,979.64,23.48
1,2021-08-12 07:50:00,3.85,NaN,NaN,10,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-12 08:00:00,3.85,NaN,NaN,60,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-12 08:10:00,3.85,NaN,NaN,58,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-12 08:20:00,3.85,NaN,NaN,58,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023-05-25 19:40:00,4.05,17.51,89.4,0,0,1015.75,NaN,19.3,19.3,19.3,NaN,1016.66,18.01
86,2023-05-25 19:50:00,4.05,17.34,88.4,0,0,1015.80,NaN,19.3,19.3,19.3,NaN,1016.71,17.88
87,2023-05-25 20:00:00,4.05,17.41,88.8,0,0,1015.90,0.0,19.3,19.3,19.3,NaN,1016.81,17.93
88,2023-05-25 20:10:00,4.05,17.17,86.4,0,0,1015.95,0.0,19.5,19.6,19.4,NaN,1016.86,17.84


Les 87 premières lignes contiennent trop de valeurs NaN , on a + de 41000 lignes donc retirer seulement 87 lignes ne va pas changer grand chose.

In [16]:
all_data.shape

(41169, 14)

In [17]:

from skimpy import *

skim(all_data)

/home/cheikh/.local/lib/python3.10/site-packages/numpy/lib/histograms.py:885: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 41169  │ │ float64     │ 11    │                                                          │
│ │ Number of columns │ 14     │ │ int64       │ 2     │                                                          │
│ └───────────────────┴────────┘ │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name             ┃ NA     ┃ NA %  ┃ mean   ┃ sd     ┃ p0    ┃ p25   ┃ p50   ┃ p75   ┃ p100 ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━━┩  │
│ │ battery (V)             │      0 │     0 │    4.1 │   0.01 │   3.6 │   4.2 │   4.2 │   4.2 │  4.2 │      ▇ │  │
│ │ dewPoint (°C)           │     13 │  0.03 │     13 │    5.4 │   -13 │   9.1 │    14 │    17 │   25 │   ▁▇▇▃ │  │
│ │ humidity (%)            │     13 │  0.03 │     71 │     13 │    13 │    63 │    72 │    80 │   98 │   ▂▆▇▃ │  │
│ │ irradiation (W/m2)      │      0 │     0 │    120 │    220 │     0 │     0 │     0 │   140 │ 1400 │  ▇▁▁   │  │
│ │ irradiation_max (W/m2)  │      0 │     0 │    140 │    240 │     0 │     0 │     0 │   180 │ 1400 │  ▇▁▁   │  │
│ │ pressure_raw (hPa)      │     13 │  0.03 │   1000 │    7.5 │   980 │  1000 │  1000 │  1000 │ 1000 │   ▂▇▃▁ │  │
│ │ rain (mm)               │     87 │  0.21 │  0.005 │  0.052 │     0 │     0 │     0 │     0 │  2.8 │   ▇    │  │
│ │ temperature (°C)        │     13 │  0.03 │     19 │    6.4 │   4.5 │    13 │    20 │    24 │   38 │ ▂▇▅▇▃  │  │
│ │ temperature_max (°C)    │     13 │  0.03 │     19 │    6.4 │   4.5 │    13 │    20 │    24 │   38 │ ▂▇▅▇▃  │  │
│ │ temperature_min (°C)    │     13 │  0.03 │     19 │    6.3 │   4.5 │    13 │    20 │    24 │   38 │ ▂▇▅▇▃  │  │
│ │ rainfall_rate_max       │  41169 │   100 │    nan │    nan │   nan │   nan │   nan │   nan │  nan │        │  │
│ │ (mm/h)                  │        │       │        │        │       │       │       │       │      │        │  │
│ │ pressure (hPa)          │     13 │  0.03 │   1000 │    7.5 │   980 │  1000 │  1000 │  1000 │ 1000 │   ▂▇▃▁ │  │
│ │ temperature_wetbulb_stu │     13 │  0.03 │     15 │    5.3 │  0.64 │    11 │    16 │    20 │   27 │  ▅▇▆▇▂ │  │
│ │ ll2011_C (°C)           │        │       │        │        │       │       │       │       │      │        │  │
│ └─────────────────────────┴────────┴───────┴────────┴────────┴───────┴───────┴───────┴───────┴──────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ time (UTC)                │       0 │          0 │                            2 │                    82338 │  │
│ └───────────────────────────┴─────────┴────────────┴──

In [18]:
n = all_data.nunique(axis=0)
 
print("unique values in each column :\n",
      n)

unique values in each column :
 time (UTC)                              41169
battery (V)                                 8
dewPoint (°C)                            2662
humidity (%)                              408
irradiation (W/m2)                        473
irradiation_max (W/m2)                    513
pressure_raw (hPa)                        939
rain (mm)                                  13
temperature (°C)                          333
temperature_max (°C)                      333
temperature_min (°C)                      331
rainfall_rate_max (mm/h)                    0
pressure (hPa)                           3800
temperature_wetbulb_stull2011_C (°C)     2389
dtype: int64


In [19]:
nb_NaN = all_data['rainfall_rate_max (mm/h)'].isna().sum()

print(nb_NaN)

41169


In [20]:
all_data.drop('rainfall_rate_max (mm/h)', axis = 1 , inplace= True)
all_data.drop('battery (V)', axis = 1 , inplace= True)

In [21]:
all_data = all_data.iloc[87:]


all_data = all_data.reset_index(drop=True)
all_data.head()

,time (UTC),dewPoint (°C),humidity (%),irradiation (W/m2),irradiation_max (W/m2),pressure_raw (hPa),rain (mm),temperature (°C),temperature_max (°C),temperature_min (°C),pressure (hPa),temperature_wetbulb_stull2011_C (°C)
0,2023-05-25 20:00:00,17.41,88.8,0,0,1015.90,0.0,19.3,19.3,19.3,1016.81,17.93
1,2023-05-25 20:10:00,17.17,86.4,0,0,1015.95,0.0,19.5,19.6,19.4,1016.86,17.84
2,2023-05-25 20:20:00,16.27,80.6,0,0,1015.95,0.0,19.7,19.7,19.7,1016.86,17.32
3,2023-05-25 20:30:00,15.27,75.6,0,0,1015.90,0.0,19.7,19.7,19.7,1016.81,16.70
4,2023-05-25 20:40:00,14.61,72.0,0,0,1015.80,0.0,19.8,19.8,19.8,1016.71,16.34


In [22]:


skim(all_data)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 41082  │ │ float64     │ 9     │                                                          │
│ │ Number of columns │ 12     │ │ int64       │ 2     │                                                          │
│ └───────────────────┴────────┘ │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name              ┃ NA  ┃ NA %  ┃ mean    ┃ sd     ┃ p0    ┃ p25   ┃ p50   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ dewPoint (°C)            │   0 │     0 │      13 │    5.4 │   -13 │   9.1 │    14 │    17 │    25 │   ▁▇▇▃ │  │
│ │ humidity (%)             │   0 │     0 │      71 │     13 │    13 │    63 │    72 │    80 │    98 │   ▂▆▇▃ │  │
│ │ irradiation (W/m2)       │   0 │     0 │     120 │    220 │     0 │     0 │     0 │   140 │   970 │  ▇▁▁   │  │
│ │ irradiation_max (W/m2)   │   0 │     0 │     140 │    240 │     0 │     0 │     0 │   180 │  1100 │  ▇▁▁▁  │  │
│ │ pressure_raw (hPa)       │   0 │     0 │    1000 │    7.5 │   990 │  1000 │  1000 │  1000 │  1000 │  ▁▅▇▂▁ │  │
│ │ rain (mm)                │   0 │     0 │   0.005 │  0.052 │     0 │     0 │     0 │     0 │   2.8 │   ▇    │  │
│ │ temperature (°C)         │   0 │     0 │      19 │    6.4 │   4.5 │    13 │    20 │    24 │    38 │ ▂▇▅▇▃  │  │
│ │ temperature_max (°C)     │   0 │     0 │      19 │    6.4 │   4.5 │    13 │    20 │    24 │    38 │ ▂▇▅▇▃  │  │
│ │ temperature_min (°C)     │   0 │     0 │      19 │    6.3 │   4.5 │    13 │    20 │    24 │    38 │ ▂▇▅▇▃  │  │
│ │ pressure (hPa)           │   0 │     0 │    1000 │    7.5 │   990 │  1000 │  1000 │  1000 │  1000 │  ▁▅▇▂▁ │  │
│ │ temperature_wetbulb_stul │   0 │     0 │      15 │    5.3 │  0.64 │    11 │    16 │    20 │    27 │  ▅▇▆▇▂ │  │
│ │ l2011_C (°C)             │     │       │         │        │       │       │       │       │       │        │  │
│ └──────────────────────────┴─────┴───────┴─────────┴────────┴───────┴───────┴───────┴───────┴───────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ time (UTC)                │       0 │          0 │                            2 │                    82164 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [23]:
all_data.dtypes

time (UTC)                               object
dewPoint (°C)                           float64
humidity (%)                            float64
irradiation (W/m2)                        int64
irradiation_max (W/m2)                    int64
pressure_raw (hPa)                      float64
rain (mm)                               float64
temperature (°C)                        float64
temperature_max (°C)                    float64
temperature_min (°C)                    float64
pressure (hPa)                          float64
temperature_wetbulb_stull2011_C (°C)    float64
dtype: object

In [47]:
scoring = {
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'R2': make_scorer(r2_score)
}

best_models = {}
param_grid = {
    'n_estimators': [50, 100, 150, 250],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

for column in all_data.columns[1:]:  
    X = all_data.drop(columns=[column, 'time (UTC)'])
    y = all_data[column]


    model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=scoring, refit='MSE', n_jobs=-1, return_train_score=True)

    grid_search.fit(X, y)

    best_mse = -grid_search.cv_results_['mean_test_MSE'][grid_search.best_index_]
    best_mae = -grid_search.cv_results_['mean_test_MAE'][grid_search.best_index_]
    best_r2 = grid_search.cv_results_['mean_test_R2'][grid_search.best_index_]


    print(f'Meilleurs paramètres pour {column}: {grid_search.best_params_}')
    print(f'Meilleur score (MSE) pour {column}: {best_mse}')
    print(f'Meilleur score (MAE) pour {column}: {best_mae}')
    print(f'Meilleur score (R2) pour {column}: {best_r2}')
    print('------------------------------------------------------')

    best_models[column] = grid_search.best_estimator_

Meilleurs paramètres pour dewPoint (°C): {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 150}
Meilleur score (MSE) pour dewPoint (°C): 0.05408426556458783
Meilleur score (MAE) pour dewPoint (°C): 0.05590460148917429
Meilleur score (R2) pour dewPoint (°C): 0.9954761691803039
------------------------------------------------------


/home/marwan/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Meilleurs paramètres pour humidity (%): {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 150}
Meilleur score (MSE) pour humidity (%): 1.0736756069902733
Meilleur score (MAE) pour humidity (%): 0.42809146930882536
Meilleur score (R2) pour humidity (%): 0.992539992792422
------------------------------------------------------
Meilleurs paramètres pour irradiation (W/m2): {'max_depth': 5, 'min_samples_split': 10, 'n_estimators': 150}
Meilleur score (MSE) pour irradiation (W/m2): 2090.7018524845653
Meilleur score (MAE) pour irradiation (W/m2): 18.03634325447381
Meilleur score (R2) pour irradiation (W/m2): 0.9454523189266542
------------------------------------------------------
Meilleurs paramètres pour irradiation_max (W/m2): {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 250}
Meilleur score (MSE) pour irradiation_max (W/m2): 1849.992021617408
Meilleur score (MAE) pour irradiation_max (W/m2): 16.948293452099225
Meilleur score (R2) pour irradiation_max (W/m2): 0.9516735

A FAIRE : 

- ensuite tester HUBER REGRESSOR et ELASTICNET puis comparer les modèles avec courbe d'identité (valeur prédite en fct de vrai valeur)

- INPUTERS (KNN et ITERATIVE)

In [24]:
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

scoring = {
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'R2': make_scorer(r2_score)
}

best_models = {}
param_grid = {
    'huberregressor__epsilon': [1.35, 1.5, 1.75, 2.0],  # Valeurs typiques d'epsilon pour HuberRegressor
    'huberregressor__alpha': [0.0001, 0.001, 0.01, 0.1],  # Valeurs typiques pour le terme de régularisation
    # Ajouter max_iter ici ne fonctionnera pas car c'est un paramètre du modèle, pas un hyperparamètre à optimiser
}

# Assume all_data est déjà défini
for column in all_data.columns[1:]:
    X = all_data.drop(columns=[column, 'time (UTC)'])
    y = all_data[column]

    # Création d'un pipeline avec mise à l'échelle et le modèle, incluant max_iter
    pipeline = Pipeline([
        ('scaler', StandardScaler()), 
        ('huberregressor', HuberRegressor(max_iter=500))  # Définir max_iter ici
    ])

    grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring=scoring, refit='MSE', n_jobs=-1, return_train_score=True)

    grid_search.fit(X, y)

    best_mse = -grid_search.cv_results_['mean_test_MSE'][grid_search.best_index_]
    best_mae = -grid_search.cv_results_['mean_test_MAE'][grid_search.best_index_]
    best_r2 = grid_search.cv_results_['mean_test_R2'][grid_search.best_index_]

    print(f'Meilleurs paramètres pour {column}: {grid_search.best_params_}')
    print(f'Meilleur score (MSE) pour {column}: {best_mse}')
    print(f'Meilleur score (MAE) pour {column}: {best_mae}')
    print(f'Meilleur score (R2) pour {column}: {best_r2}')
    print('------------------------------------------------------')

    best_models[column] = grid_search.best_estimator_


Meilleurs paramètres pour dewPoint (°C): {'huberregressor__alpha': 0.1, 'huberregressor__epsilon': 2.0}
Meilleur score (MSE) pour dewPoint (°C): 0.24986275835840618
Meilleur score (MAE) pour dewPoint (°C): 0.16973228785119315
Meilleur score (R2) pour dewPoint (°C): 0.9798059117618084
------------------------------------------------------
Meilleurs paramètres pour humidity (%): {'huberregressor__alpha': 0.1, 'huberregressor__epsilon': 2.0}
Meilleur score (MSE) pour humidity (%): 8.529209867788046
Meilleur score (MAE) pour humidity (%): 1.583668643151458
Meilleur score (R2) pour humidity (%): 0.9392175634862394
------------------------------------------------------
Meilleurs paramètres pour irradiation (W/m2): {'huberregressor__alpha': 0.1, 'huberregressor__epsilon': 2.0}
Meilleur score (MSE) pour irradiation (W/m2): 2097.579085686598
Meilleur score (MAE) pour irradiation (W/m2): 15.393097585393889
Meilleur score (R2) pour irradiation (W/m2): 0.9295412911664812
--------------------------